In [ ]:
import os
import pandas as pd
import json

# Define la ruta de la carpeta y el archivo JSON
folder_path = 'Datos/Happiness'
json_path = 'data_country.json'

# Define los mapeos de columnas
column_mappings = {
    'Country': ['Country', 'Country or region','Country name'],
    'Rank': ['Happiness Rank', 'Overall rank', 'Happiness.Rank','Rank'],
    'Score': ['Happiness Score', 'Score', 'Happiness.Score','Ladder score','Happiness score'],
    'GDP per capita': ['Economy (GDP per Capita)', 'Economy..GDP.per.Capita.', 'GDP per capita','Logged GDP per capita','Explained by: GDP per capita'],
    'Family': ['Family', 'Social support','Explained by: Social support'],
    'Health (Life Expectancy)': ['Health (Life Expectancy)', 'Health..Life.Expectancy.', 'Healthy life expectancy','Explained by: Healthy life expectancy'],
    'Freedom': ['Freedom', 'Freedom to make life choices','Explained by: Freedom to make life choices'],
    'Perceptions of corruption': ['Trust (Government Corruption)', 'Trust..Government.Corruption.', 'Perceptions of corruption','Explained by: Perceptions of corruption'],
    'Generosity': ['Generosity','Explained by: Generosity']
}

# Crear un diccionario inverso para los nombres alternativos
alternate_names_to_official = {}
with open(json_path, 'r', encoding='utf-8') as file:
    data_country = json.load(file)
    for country, info in data_country.items():
        for alt_name in info.get('AlternateNames', []):
            alternate_names_to_official[alt_name] = country

# Función para estandarizar los nombres de las columnas y eliminar las no deseadas
def standardize_columns(df):
    standardized_columns = {}
    for standard_col, possible_cols in column_mappings.items():
        for col in possible_cols:
            if col in df.columns:
                standardized_columns[col] = standard_col
                break
    df = df.rename(columns=standardized_columns)
    # Eliminar columnas que no están en los mapeos
    columns_to_keep = list(standardized_columns.values())
    df = df[columns_to_keep]
    return df

# Lista para almacenar los dataframes
dataframes = []

# DataFrame consolidado inicial
consolidated_df = pd.DataFrame()

# Iterar sobre los archivos en la carpeta
for filename in os.listdir(folder_path):
    if filename.startswith('Happiness') and filename.endswith('.csv'):
        # Extraer el año del nombre del archivo
        year = int(''.join(filter(str.isdigit, filename)))
        
        # Leer el archivo CSV
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        # Eliminar filas vacías
        df.dropna(how='all', inplace=True)
        
        # Estandarizar las columnas y eliminar las no deseadas
        df = standardize_columns(df)
        
        # Asegurarse de que la columna Rank sea un entero
        if 'Rank' in df.columns:
            df['Rank'] = df['Rank'].astype(int)
        else:
            df['Rank'] = range(1, len(df) + 1)
        
        # Asegurarse de que los valores de 'Country' sean cadenas y manejar valores nulos
        df['Country'] = df['Country'].astype(str).fillna('')
        
        # Reemplazar nombres alternativos por nombres oficiales
        df['Country'] = df['Country'].apply(lambda x: alternate_names_to_official.get(x, x))
        
        # Filtrar las filas basadas en el archivo JSON
        df = df[df['Country'].isin(data_country)]
        
        # Redondear las columnas numéricas a 3 dígitos si existen
        numeric_columns = ['Rank', 'Score', 'GDP per capita', 'Family', 
                           'Health (Life Expectancy)', 'Freedom', 
                           'Perceptions of corruption', 'Generosity']
        for col in numeric_columns:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)
        
        # Renombrar las columnas para incluir el año
        df = df.rename(columns={col: f"{col} ({year})" for col in df.columns if col != 'Country'})
        
        # Si el DataFrame consolidado está vacío, inicializarlo con el primer DataFrame
        if consolidated_df.empty:
            consolidated_df = df.set_index('Country')
        else:
            # Alinear los datos del nuevo DataFrame con el DataFrame consolidado
            df = df.set_index('Country')
            consolidated_df = consolidated_df.join(df, how='outer')
        
# Ordenar el DataFrame combinado por el nombre de la columna "Country"
consolidated_df.sort_values(by=['Country'], inplace=True)

# Guardar el dataframe consolidado en un nuevo archivo CSV
consolidated_df.to_csv('consolidado_happiness_v2.csv')

print("Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness_v2.csv.")

Consolidación completa. Los datos consolidados se han guardado en consolidado_happiness_v2.csv.


In [93]:
# Verificar el directorio actual de trabajo
current_directory = os.getcwd()
print(f"Directorio actual: {current_directory}")

Directorio actual: c:\Users\ProjectSupport\Documents\David\Cursos\Master Data Science\Proyecto
